# Action Recognition with ResNet

In [ ]:
from torch.utils.data import DataLoader
from main import Stanford40Dataset
from torchvision import transforms
import torch

## Prepare image transformation and constants

In [ ]:
# Transforms and constant definition
# Define transformations (resize, normalization, etc.)
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8
    ),
    transforms.RandomGrayscale(0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Paths to the image directory (data/train and data/test)
data_dir = 'data'
LR = 1e-3
BATCH_SIZE = 196
EPOCHS = 50
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Load dataset

In [ ]:
from torch.utils.data import random_split

# Instantiate the dataset and data loaders
train_dataset = Stanford40Dataset(data_dir=data_dir, split='train', transform=transform)
test_dataset = Stanford40Dataset(data_dir=data_dir, split='test', transform=transform)

numTrainSamples = int(len(train_dataset) * TRAIN_SPLIT)
numValSamples = int(len(train_dataset) * VAL_SPLIT)
(trainData, valData) = random_split(train_dataset,
                                    [numTrainSamples, numValSamples],
                                    generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(trainData, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(valData, batch_size=BATCH_SIZE)


## Instantiate ResNet model

In [ ]:
from torch import optim, nn
from CustomResNet import CustomResNet

#Define model
# Create an instance of the model class and allocate it to the device
model = CustomResNet().to(device)

print(model)

optimizer = optim.Adam(model.parameters(), lr=LR)
loss_criteria = nn.CrossEntropyLoss()


## Training loop

In [ ]:
from TrainTestUtils import train, validate, plot_loss

epoch_nums = []
training_loss = []
validation_loss = []

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    # Call the train function
    train_loss = train(model, device, loss_criteria, train_loader, optimizer, epoch)
    # Call the validation function
    val_loss = validate(model, device, loss_criteria, val_loader)

    # Track losses and epochs
    epoch_nums.append(epoch)
    training_loss.append(train_loss)
    validation_loss.append(val_loss)
    
plot_loss(epoch_nums, training_loss, validation_loss)

## Testing (performing predictions)

In [ ]:
from TrainTestUtils import make_predictions, compute_accuracy

#Testing
print("Evaluating on the test set...")
truelabels, predictions = make_predictions(model, test_loader, device)

# Compute and display accuracy
accuracy = compute_accuracy(truelabels, predictions)
print(f"Test Set Accuracy: {accuracy:.2f}%")